# Preprocess dataset by expert knowledge

### Load original dataset from csv file

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import category_encoders as ce

<IPython.core.display.Javascript object>

In [3]:
df = pd.read_csv("VN_housing_dataset.csv")

print(f"{len(df)} instances")
display(df.head())

82497 instances


,Unnamed: 0,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,0.0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,1.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2.0,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,3.0,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,4.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"


<IPython.core.display.Javascript object>

### Drop all nan value, unnamed columns, sale date, address

In [4]:
df = df.drop(columns=["Unnamed: 0", "Ngày", "Địa chỉ"])
df = df.dropna()

<IPython.core.display.Javascript object>

### Filter out price (million/m2)

In [5]:
df = df[df["Giá/m2"].str.contains("triệu/m²")]
df["Giá/m2"] = [float(x.replace(",", ".").split()[0]) for x in list(df["Giá/m2"])]

<IPython.core.display.Javascript object>

### Filter out area(s), length(l), width(w)
Remove the lands that have a 25% difference in area from length * width.

In [6]:
eps_s = 0.25
df["Diện tích"] = [float(x.split()[0]) for x in list(df["Diện tích"])]
df["Dài"] = [float(x.split()[0]) for x in list(df["Dài"])]
df["Rộng"] = [float(x.split()[0]) for x in list(df["Rộng"])]
lower = df[df["Dài"] * df["Rộng"] * (1 - eps_s) > df["Diện tích"]].index
df = df.drop(lower)
upper = df[df["Dài"] * df["Rộng"] * (1 + eps_s) < df["Diện tích"]].index
df = df.drop(upper)

<IPython.core.display.Javascript object>

### Filter out number of bedrooms (nb)
Drop out all instances i such that: nb >= 10

In [7]:
df = df.drop(df[df["Số phòng ngủ"] == "nhiều hơn 10 phòng"].index)
df["Số phòng ngủ"] = [float(x.split()[0]) for x in list(df["Số phòng ngủ"])]

<IPython.core.display.Javascript object>

### Filter out number of floor (nf)
Drop out all instances i such that: nf >= 10


In [8]:
df = df.drop(df[df["Số tầng"] == "Nhiều hơn 10"].index)
df["Số tầng"] = [float(x) for x in list(df["Số tầng"])]
df = df.drop(df[df["Số tầng"] > 10].index)

<IPython.core.display.Javascript object>

# Review categorical variables

In [9]:
def review(df, attribute):
    print("=" * 100)
    print(len(set(list(df[attribute]))))
    for x in set(list(df[attribute])):
        print(f"{x}: {list(df[attribute]).count(x)}")

<IPython.core.display.Javascript object>

In [10]:
review(df, "Giấy tờ pháp lý")
review(df, "Loại hình nhà ở")
review(df, "Huyện")
review(df, "Quận")

3
Đang chờ sổ: 77
Giấy tờ khác: 64
Đã có sổ: 9990
4
Nhà ngõ, hẻm: 7273
Nhà phố liền kề: 356
Nhà biệt thự: 173
Nhà mặt phố, mặt tiền: 2329
232
Phường Vạn Phúc: 103
Phường Thịnh Liệt: 75
Phường Đông Ngạc: 8
Phường Ngọc Thụy: 98
Phường Đội Cấn: 115
Xã Khánh Hà: 1
Phường Mai Động: 39
Phường Tràng Tiền: 2
Phường Định Công: 165
Thị trấn Sóc Sơn: 1
Phường Dịch Vọng Hậu: 100
Xã Đông La: 16
Phường Phúc La: 42
Xã Phương Trung: 1
Phường Kim Liên: 37
Phường Thanh Nhàn: 111
Phường Tứ Liên: 45
Thị trấn Trạm Trôi: 8
Thị trấn Chúc Sơn: 2
Phường Hàng Bột: 76
Phường Phú Đô: 20
Phường Mỹ Đình 2: 68
Phường Bùi Thị Xuân: 7
Phường Cầu Dền: 12
Phường Tây Mỗ: 30
Phường Lý Thái Tổ: 2
Phường Minh Khai: 157
Phường Quảng An: 31
Phường Khương Đình: 227
Phường Mễ Trì: 36
Xã Nam Hồng: 1
Phường Văn Chương: 33
Xã Bích Hòa: 3
Xã Đa Tốn: 1
Phường Trương Định: 92
Phường Vĩnh Phúc: 75
Xã Phú Cát: 2
Phường Trúc Bạch: 13
Phường Khâm Thiên: 72
Xã Bắc Hồng: 1
Phường Hà Cầu: 92
Xã Thanh Liệt: 37
Phường Nguyễn Du: 1
Phường Phú 

<IPython.core.display.Javascript object>

### Rename columns: Vietnamese to English

In [11]:
df = df.reset_index(drop=True)
df = df.rename(
    columns={
        "Quận": "district",
        "Huyện": "ward",
        "Loại hình nhà ở": "type_of_housing",
        "Giấy tờ pháp lý": "legal_paper",
        "Số tầng": "floor_num",
        "Số phòng ngủ": "bedroom_num",
        "Diện tích": "area",
        "Dài": "length",
        "Rộng": "width",
        "Giá/m2": "price/m2",
    }
)
print(f"{len(df)} instances")
display(df.head())

10131 instances


,district,ward,type_of_housing,legal_paper,floor_num,bedroom_num,area,length,width,price/m2
0,Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,10.0,4.0,65.00
1,Quận Long Biên,Phường Bồ Đề,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,52.0,12.0,4.2,93.27
2,Quận Hà Đông,Phường La Khê,"Nhà mặt phố, mặt tiền",Đã có sổ,5.0,5.0,90.0,18.0,5.0,108.89
3,Quận Nam Từ Liêm,Phường Tây Mỗ,"Nhà ngõ, hẻm",Đã có sổ,4.0,3.0,32.0,6.6,4.5,60.94
4,Huyện Thanh Trì,Xã Tả Thanh Oai,"Nhà ngõ, hẻm",Đã có sổ,3.0,2.0,42.0,11.0,4.0,29.76


<IPython.core.display.Javascript object>

### Ordinal encoding: type_of_housing, legal_paper

In [12]:
df["type_of_housing"] = df.type_of_housing.map(
    {
        "Nhà biệt thự": 4.0,
        "Nhà mặt phố, mặt tiền": 3.0,
        "Nhà phố liền kề": 2.0,
        "Nhà ngõ, hẻm": 1.0,
    }
)
df["legal_paper"] = df.legal_paper.map(
    {"Đã có sổ": 3.0, "Đang chờ sổ": 2.0, "Giấy tờ khác": 1.0}
)

<IPython.core.display.Javascript object>

### Nominal encoding: district, ward

In [13]:
df = ce.OneHotEncoder(cols=["district", "ward"]).fit_transform(df)

<IPython.core.display.Javascript object>

### Export processed dataset to csv file

In [14]:
df.to_csv("VN_housing_dataset_processed_expert_knowledge.csv", index=False)

<IPython.core.display.Javascript object>

### Load processed dataset from csv file
Review processed dataset

In [15]:
df = pd.read_csv("VN_housing_dataset_processed_expert_knowledge.csv")

print(f"{len(df)} instances")
display(df.head())

10131 instances


,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,...,ward_231,ward_232,type_of_housing,legal_paper,floor_num,bedroom_num,area,length,width,price/m2
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1.0,3.0,4.0,4.0,40.0,10.0,4.0,65.00
1,0,1,0,0,0,0,0,0,0,0,...,0,0,1.0,3.0,5.0,4.0,52.0,12.0,4.2,93.27
2,0,0,1,0,0,0,0,0,0,0,...,0,0,3.0,3.0,5.0,5.0,90.0,18.0,5.0,108.89
3,0,0,0,1,0,0,0,0,0,0,...,0,0,1.0,3.0,4.0,3.0,32.0,6.6,4.5,60.94
4,0,0,0,0,1,0,0,0,0,0,...,0,0,1.0,3.0,3.0,2.0,42.0,11.0,4.0,29.76


<IPython.core.display.Javascript object>